# Averaged Artifact Substraction with EEG Data
## First step prepare Jupyter Notebook

In [1]:
import os
from FACET.Facet import Facet
from loguru import logger
import sys

os.getcwd()
os.chdir(os.path.dirname(os.path.abspath("__file__")))

bids_path = "F:\EEG Datasets\openneuro\FMRIWITHMOTION"
export_bids_path = bids_path

# configure logger
logger.remove()
logger.add(sys.stderr, level="INFO")
logger.add("Facet.log", level="DEBUG")

window_size = 25
upsampling_factor = 1
artifact_to_trigger_offset_in_seconds = -6
relative_window_position = -0.5
moosmann_motion_threshold = 0.8
event_id_description_pairs={'trigger':1}
regex_trigger_annotation_filter = r'\bResponse\b' # Annotations with the description 'trigger' are considered as triggers
unwanted_bad_channels = ['EKG', 'EMG', 'EOG', 'ECG'] # Channels with these names are considered as bad channels and not considered in the processing
evaluation_measures=["SNR", "RMS", "RMS2", "MEDIAN"]

f = Facet()


## Second Step import EEG Data

In [2]:
f.import_EEG(path=bids_path,fmt="bids",upsampling_factor=upsampling_factor,artifact_to_trigger_offset=artifact_to_trigger_offset_in_seconds, bads=['EKG', 'EMG', 'EOG', 'ECG'], subject="xp101", session=None,task="eegfmriNF")
eeg = f.get_EEG()


2024-03-26 10:49:48.146 | INFO     | FACET.Facet:import_EEG:18 - Importing EEG from F:\EEG Datasets\openneuro\FMRIWITHMOTION


FileNotFoundError: [WinError 3] Das System kann den angegebenen Pfad nicht finden: 'F:\\EEG Datasets\\openneuro\\FMRIWITHMOTION\\sub-xp101'

## Third Step Preprocessing

In [ ]:
f.pre_processing()


## Fourth Step find triggers

In [ ]:

f.find_triggers(regex_trigger_annotation_filter) # Using Niazys data
eeg = f.get_EEG()


## Fourth step apply AAS

In [ ]:
f.apply_AAS()

## Fifth step Remove Artifacts

In [ ]:
f.remove_artifacts( )
eeg = f.get_EEG()



## Sixth step Post-Processing

In [ ]:
f.post_processing()

## Seventh display the processed eeg data

In [ ]:
f.plot_EEG(title="after lowpass")

## Eighth Evaluate Results and compare with other Methods

In [ ]:
f.find_triggers(regex_trigger_annotation_filter) 
eeg_python = f.get_EEG()
f.add_to_evaluate(eeg_python, name="numpy_matrix") # Matrix will result in >1.0 RMS2 because of python mne inaccuracies

#moosmann
f.import_EEG(path=bids_path,fmt="bids",artifact_to_trigger_offset=artifact_to_trigger_offset_in_seconds,upsampling_factor=upsampling_factor, bads=unwanted_bad_channels, subject="xp101", session=None,task="eegfmriNF")
f.pre_processing()
f.find_triggers(regex_trigger_annotation_filter)
f.apply_Moosmann(file_path="headmotiondata_209s.tsv", threshold=moosmann_motion_threshold)
f.remove_artifacts()
f.downsample()
f.lowpass(40)
f.find_triggers(regex_trigger_annotation_filter)
f.add_to_evaluate(f.get_EEG(), name="Moosmann")



results = f.evaluate(plot=True,measures=["SNR", "RMS", "RMS2", "MEDIAN"])
logger.info(results)

f.export_EEG(fmt="bids", event_id=event_id_description_pairs, path=export_bids_path)

